In [80]:
import numpy as np
import pandas as pd
from scipy.special import expit
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy.spatial import distance
import statsmodels.api as sm
import warnings
import numpy as np
from scipy.optimize import differential_evolution
from scipy.stats import rankdata
from sklearn.cross_decomposition import CCA
import math
import copy
from math import log
from collections import Counter
import pickle
warnings.filterwarnings("ignore")

In [82]:
with open('relation_matrix_dic.pkl', 'rb') as f:
    relation_matrix_dic = pickle.load(f)

error_array = np.load('error.npy')

# define a new adjacency_matrix_dic: adjacency_matrix_dic[i] = relation_matrix_dic[i].iloc[0:20, 0:20]
adjacency_matrix_dic = {i: relation_matrix_dic[i].iloc[0:20, 0:20] for i in relation_matrix_dic}

K = 3

# Read beta back from the file
beta = np.load('beta.npy')
print("beta:", beta)

beta: [3.0143364  4.21106128 4.49056789 6.84433191 4.38055757 4.59035439
 5.62650526 4.348927   3.26627828 2.23006741]


In [84]:
# Parameters
K = 12

# Compute total length
total_length = 1 + K + K + math.comb(K, 2)
total_length

91

In [86]:
new_beta = np.zeros(total_length)

# Assign values based on the provided instructions
new_beta[0:4] = beta[0:4]            # Copy the first four elements unchanged
new_beta[10:13] = beta[4:7]          # Copy beta[4:6] into new_beta[11:13]
new_beta[20:22] = beta[7:9]          # Copy beta[7:8] into new_beta[20:21]

# Print the new_beta array
print("new_beta:", new_beta)

beta = new_beta

new_beta: [3.0143364  4.21106128 4.49056789 6.84433191 0.         0.
 0.         0.         0.         0.         4.38055757 4.59035439
 5.62650526 0.         0.         0.         0.         0.
 0.         0.         4.348927   3.26627828 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.        ]


In [88]:
for i in range(K+1):
    print(f"Position {i}: beta = {beta[i]}")

idx = K + 1

for i in range(1, K+1):
    for j in range(1, K+1):
        if i <= j:
            print(f"Position ({i},{j}): beta = {beta[idx]}")
            idx += 1

Position 0: beta = 3.0143363957683738
Position 1: beta = 4.211061281023643
Position 2: beta = 4.490567890019025
Position 3: beta = 6.84433191283132
Position 4: beta = 0.0
Position 5: beta = 0.0
Position 6: beta = 0.0
Position 7: beta = 0.0
Position 8: beta = 0.0
Position 9: beta = 0.0
Position 10: beta = 4.3805575746037055
Position 11: beta = 4.590354387978943
Position 12: beta = 5.626505257258666
Position (1,1): beta = 0.0
Position (1,2): beta = 0.0
Position (1,3): beta = 0.0
Position (1,4): beta = 0.0
Position (1,5): beta = 0.0
Position (1,6): beta = 0.0
Position (1,7): beta = 0.0
Position (1,8): beta = 4.348926998816991
Position (1,9): beta = 3.2662782844053506
Position (1,10): beta = 0.0
Position (1,11): beta = 0.0
Position (1,12): beta = 0.0
Position (2,2): beta = 0.0
Position (2,3): beta = 0.0
Position (2,4): beta = 0.0
Position (2,5): beta = 0.0
Position (2,6): beta = 0.0
Position (2,7): beta = 0.0
Position (2,8): beta = 0.0
Position (2,9): beta = 0.0
Position (2,10): beta = 0.0

In [90]:
# define a function named "counting_treatment"
def counting_treatment(adjacency_matrix, A, K):

    # adjacency_matrix is a adjacency matrix dataframe with length 20 * 20 known
    # A is a np.array with length 20, each element 1,2,...,K known corresponding to each node in the network

    # a empty df named counting_treatment with column named: 0,1,...,K, (1,1), ..., (1,K), (2,2), ..., (2,K),...(K,K)

    # add different numbers of different nodes and different edges. For the column 0, directly give 1.

    # Define column names
    basic_columns = list(range(0, K + 1))  # Including column 0
    edge_columns = [(i, j) for i in range(1, K + 1) for j in range(i, K + 1)]
    columns = basic_columns + edge_columns
    
    # Create an empty DataFrame
    counting_treatment_df = pd.DataFrame(columns=columns)
    counting_treatment_df.loc[0] = 0  # Initialize all counts to 0
    
    # Column 0 gets the value of 1 as per the description
    counting_treatment_df.at[0, 0] = 1

    # Count number of nodes with each treatment
    for treatment in range(1, K + 1):
        counting_treatment_df.at[0, treatment] = sum(A == treatment)
    
    # Count the number of edges connecting nodes of different treatments
    for i in range(20):
        for j in range(i + 1, 20):  # Iterate only for j > i to avoid double-counting
            if adjacency_matrix.iloc[i, j] != 0:  # If there's an edge between node i and node j
                treatment_i = A[i]
                treatment_j = A[j]
                # Use sorted tuple to represent undirected edge
                edge = tuple(sorted((treatment_i, treatment_j)))
                if edge in counting_treatment_df.columns:
                    counting_treatment_df.at[0, edge] += 1
                
    return counting_treatment_df

# generate_outcome 

In [93]:
def generate_outcome(adjacency_matrix, A, K, beta, error):
    # Generate the treatment matrix using the counting_treatment function
    counting_treatment_df = counting_treatment(adjacency_matrix, A, K)

    # Convert the treatment matrix to a NumPy array for easier calculation
    counting_treatment_array = counting_treatment_df.values.flatten()

    # Ensure that beta and counting_treatment_array have the same length
    if len(beta) != len(counting_treatment_array):
        raise ValueError("Length of beta does not match the number of columns in counting_treatment.")

    # Compute the outcome Y
    Y = np.dot(counting_treatment_array, beta) + error

    # Return the results
    return Y, counting_treatment_df

# optimization for one network adjacency_matrix

In [96]:
import numpy as np

def get_optimal_experiment(adjacency_matrix, K, beta, error, prob):
    # Find the optimal treatment arm for the main effect from beta values
    K_main_optimal = 1  # Get the index of the maximum value from beta[1:K+1] (adjusted to start from 1)

    # Set the initial treatment for all nodes to K_main_optimal
    A_optimal = np.full(adjacency_matrix.shape[0], K_main_optimal)
    A = A_optimal.copy()

    no_improve = 0
    Y_optimal, counting_treatment_df = generate_outcome(adjacency_matrix, A_optimal, K, beta, error)
    print(f"Epoch 0: Reward = {Y_optimal}")

    for i in range(100):
        for j in range(adjacency_matrix.shape[0]):
            # Store the best reward and treatment configuration found for this node update
            best_Y = -np.inf
            best_A = A.copy()

            for k in range(1, K + 1):
                if A[j] != k:
                    A_candidate = A.copy()  # Make a copy of A to modify
                    A_candidate[j] = k  # Set A[j] to k in the candidate
                    Y, counting_treatment_df = generate_outcome(adjacency_matrix, A_candidate, K, beta, error)

                    if Y > best_Y:
                        best_Y = Y
                        best_A = A_candidate

            # Update the optimal reward and treatment configuration if an improvement is found
            if best_Y > Y_optimal:
                A = best_A
                Y_optimal = best_Y
                A_optimal = A.copy()
                no_improve = 0
            else:
                # Update A with the best_A with a certain probability, otherwise leave A unchanged
                v = best_Y / 400
                prob = math.exp(- v * t)
                if np.random.rand() < prob:
                    A = best_A
                no_improve += 1

            # Generate the outcome with the updated configuration A
            Y, counting_treatment_df = generate_outcome(adjacency_matrix, A, K, beta, error)
            print(f"Epoch {i * adjacency_matrix.shape[0] + (j + 1)}: Reward = {Y}")
            print(f"Epoch {i * adjacency_matrix.shape[0] + (j + 1)}: Optimal reward = {Y_optimal}")

            # Break out of both loops if no improvement for consecutive nodes
            if no_improve > adjacency_matrix.shape[0] :
                return Y_optimal, A_optimal

    return Y_optimal, A_optimal


# initial experiment 0: for a selected vector treatments, we get rewards

In [98]:
# Initialize empty lists to store A and reward over time
optimal_A_time = []
optimal_reward_time = []

generation_num = 100
epoch = 100

# Iterate over the relation_matrix_dic
for t in range(len(adjacency_matrix_dic)):

    print("time:", t)

    adjacency_matrix = adjacency_matrix_dic[t]
    error = error_array[t]
    
    # Get optimal outcome Y and experiment assignment A 
    Y, A = get_optimal_experiment(adjacency_matrix, K, beta, error,0.1)

    print("optimal treatment:", A)
    print("optimal reward:", Y)
    
    # Store the optimal assignment and reward
    optimal_A_time.append(A)
    optimal_reward_time.append(Y)

time: 0
Epoch 0: Reward = 88.14914808686329
Epoch 1: Reward = 127.42735679400957
Epoch 1: Optimal reward = 127.42735679400957
Epoch 2: Reward = 184.1012734964238
Epoch 2: Optimal reward = 184.1012734964238
Epoch 3: Reward = 232.07733620120405
Epoch 3: Optimal reward = 232.07733620120405
Epoch 4: Reward = 230.36167983419472
Epoch 4: Optimal reward = 232.07733620120405
Epoch 5: Reward = 282.68666953779194
Epoch 5: Optimal reward = 282.68666953779194
Epoch 6: Reward = 267.9242321743317
Epoch 6: Optimal reward = 282.68666953779194
Epoch 7: Reward = 294.155659885027
Epoch 7: Optimal reward = 294.155659885027
Epoch 8: Reward = 316.0381605969053
Epoch 8: Optimal reward = 316.0381605969053
Epoch 9: Reward = 320.52495331351565
Epoch 9: Optimal reward = 320.52495331351565
Epoch 10: Reward = 316.313892032492
Epoch 10: Optimal reward = 320.52495331351565
Epoch 11: Reward = 320.8006847491023
Epoch 11: Optimal reward = 320.8006847491023
Epoch 12: Reward = 294.8449884739937
Epoch 12: Optimal reward =

In [100]:
optimal_reward_time_greedy = np.array(optimal_reward_time)

In [101]:
# Save to a .npy file
np.save('optimal_reward_time_greedy.npy', optimal_reward_time_greedy)